In [1]:
import pandas as pd
import numpy as np
import re

## We want to unify the MovieLens movieId and IMDBid for our database

### Load Completed IMDB Blockbuster movie dataset from 1_Resolve_IMDB_Disney_Movie_Title

In [2]:
#Load the complete IMDB dataset
complete_df = pd.read_csv('output/complete_Disney_movie.csv')
complete_df['Year'] = complete_df['Year'].str.split('–', 1).str[0]
complete_df['Genre'] = complete_df['Genre'].str.replace(',', '|')
complete_df['Genre'] = complete_df['Genre'].str.replace(' ', '')

complete_df['Director'] = complete_df['Director'].str.replace(',', '|')
#complete_df['Director'] = complete_df['Director'].str.replace(' ', '')

complete_df['Writer'] = complete_df['Writer'].str.replace(',', '|')
#complete_df['Writer'] = complete_df['Writer'].str.replace(' ', '')

complete_df['Actors'] = complete_df['Actors'].str.replace(',', '|')
#complete_df['Actors'] = complete_df['Actors'].str.replace(' ', '')

complete_df['Language'] = complete_df['Language'].str.replace(',', '|')
complete_df['Language'] = complete_df['Language'].str.replace(' ', '')

complete_df['Country'] = complete_df['Country'].str.replace(',', '|')
#complete_df['Country'] = complete_df['Country'].str.replace(' ', '')


#complete_df['imdbVotes'] = complete_df['imdbVotes'].str.replace(',', '')

complete_df.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,Country,Awards,Metascore,IMDBRating,imdbVotes,imdbID,total_gross,inflation_adjusted_gross
0,101 Dalmatians,1996,G,27 Nov 1996,103.0,Adventure|Comedy|Crime,Stephen Herek,Dodie Smith| John Hughes,Glenn Close| Jeff Daniels| Joely Richardson,English|Spanish,United States| United Kingdom,Nominated for 1 BAFTA Film Award3 wins & 10 no...,49.0,5.7,109762,tt0115433,153000000.0,1.386569e+09
1,101 Dalmatians,1996,G,27 Nov 1996,103.0,Adventure|Comedy|Crime,Stephen Herek,Dodie Smith| John Hughes,Glenn Close| Jeff Daniels| Joely Richardson,English|Spanish,United States| United Kingdom,Nominated for 1 BAFTA Film Award3 wins & 10 no...,49.0,5.7,109762,tt0115433,136189294.0,2.352021e+08
2,102 Dalmatians,2000,G,22 Nov 2000,100.0,Adventure|Comedy|Family,Kevin Lima,Dodie Smith| Kristen Buckley| Brian Regan,Glenn Close| Gérard Depardieu| Ioan Gruffudd,English,United States| United Kingdom,Nominated for 1 Oscar. 1 win & 8 nominations t...,35.0,4.9,37045,tt0211181,66941559.0,1.053377e+08
3,1492: Conquest of Paradise,1992,PG-13,09 Oct 1992,154.0,Adventure|Biography|Drama,Ridley Scott,Rose Bosch,Gérard Depardieu| Armand Assante| Sigourney We...,English|Spanish|Latin|Arabic,United Kingdom| France| Spain,2 nominations,47.0,6.4,29990,tt0103594,7099531.0,1.371176e+07
4,"20,000 Leagues Under the Sea",1954,G,20 Jul 1955,127.0,Adventure|Drama|Family,Richard Fleischer,Earl Felton| Jules Verne,Kirk Douglas| James Mason| Paul Lukas,English,United States,Won 2 Oscars. 5 wins & 3 nominations total,83.0,7.2,33152,tt0046672,28200000.0,2.840652e+08


In [3]:
complete_df['Year'] = complete_df['Year'].astype(int)

### Load MovieLens movies dataset

In [4]:
#load Movie Lens movies dataset
ml_df = pd.read_csv('input/movies.csv')
ml_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
#ml_df['title'] = ml_df['title'].str.replace('.','')
ml_df['movie_title'] = ml_df['title'].str.split(' \(', 1).str[0]
ml_df['movie_title'] = ml_df['movie_title'].str.replace('.','')
ml_df['year'] = ml_df['title'].str.extract('(\(\d{4}\)$)')
ml_df['year'] = ml_df['year'].str.replace('\)','')
ml_df['year'] = ml_df['year'].str.replace('\(','')

In [6]:
#Can't parse year to int as there are some null values
ml_df[ml_df['year'].isnull()]

,movieId,title,genres,movie_title,year
8463,25936,"Babe Ruth Story, The (1948)",Drama,"Babe Ruth Story, The",NaN
8602,26137,"Heroes of Telemark, The (1965)",Action|Drama|War,"Heroes of Telemark, The",NaN
8910,26628,Jack's Back (1988),Crime|Horror|Mystery|Thriller,Jack's Back,NaN
9120,27189,After the Rain (Ame agaru) (1999),Action|Drama,After the Rain,NaN
9763,32497,Love Letter (1995),Drama|Romance,Love Letter,NaN
...,...,...,...,...,...
62071,207714,Tales of Found Footage,(no genres listed),Tales of Found Footage,NaN
62104,207884,Enduring Destiny,(no genres listed),Enduring Destiny,NaN
62285,208597,Punk the Capital: Building a Sound Movement,Documentary,Punk the Capital: Building a Sound Movement,NaN
62326,208763,Yosemite: The Fate of Heaven,(no genres listed),Yosemite: The Fate of Heaven,NaN


In [7]:
ml_df[ml_df.movie_title.str.startswith('101 Dalmatians')]

,movieId,title,genres,movie_title,year
1331,1367,101 Dalmatians (1996),Adventure|Children|Comedy,101 Dalmatians,1996
1996,2085,101 Dalmatians (One Hundred and One Dalmatians...,Adventure|Animation|Children,101 Dalmatians,1961
24273,121099,101 Dalmatians II: Patch's London Adventure (2...,Animation|Children,101 Dalmatians II: Patch's London Adventure,2003


In [8]:
#Fill Null year with -1
ml_df['year'].fillna("-1", inplace = True)
#Parse year as int
ml_df['year'] = ml_df['year'].astype(int)

In [9]:
#genres na?
ml_df.loc[ml_df['genres'].isna()]

,movieId,title,genres,movie_title,year


In [10]:
#Merge the IMDB complete dataset and MovieLens dataset on movie_title & year ONLY
merged_disney_df =pd.merge(complete_df, ml_df, how='left', left_on=['Title', 'Year'], right_on=['movie_title', 'year'])
merged_disney_df.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,...,IMDBRating,imdbVotes,imdbID,total_gross,inflation_adjusted_gross,movieId,title,genres,movie_title,year
0,101 Dalmatians,1996,G,27 Nov 1996,103.0,Adventure|Comedy|Crime,Stephen Herek,Dodie Smith| John Hughes,Glenn Close| Jeff Daniels| Joely Richardson,English|Spanish,...,5.7,109762,tt0115433,153000000.0,1.386569e+09,1367.0,101 Dalmatians (1996),Adventure|Children|Comedy,101 Dalmatians,1996.0
1,101 Dalmatians,1996,G,27 Nov 1996,103.0,Adventure|Comedy|Crime,Stephen Herek,Dodie Smith| John Hughes,Glenn Close| Jeff Daniels| Joely Richardson,English|Spanish,...,5.7,109762,tt0115433,136189294.0,2.352021e+08,1367.0,101 Dalmatians (1996),Adventure|Children|Comedy,101 Dalmatians,1996.0
2,102 Dalmatians,2000,G,22 Nov 2000,100.0,Adventure|Comedy|Family,Kevin Lima,Dodie Smith| Kristen Buckley| Brian Regan,Glenn Close| Gérard Depardieu| Ioan Gruffudd,English,...,4.9,37045,tt0211181,66941559.0,1.053377e+08,3991.0,102 Dalmatians (2000),Children|Comedy,102 Dalmatians,2000.0
3,1492: Conquest of Paradise,1992,PG-13,09 Oct 1992,154.0,Adventure|Biography|Drama,Ridley Scott,Rose Bosch,Gérard Depardieu| Armand Assante| Sigourney We...,English|Spanish|Latin|Arabic,...,6.4,29990,tt0103594,7099531.0,1.371176e+07,8905.0,1492: Conquest of Paradise (1992),Adventure|Drama,1492: Conquest of Paradise,1992.0
4,"20,000 Leagues Under the Sea",1954,G,20 Jul 1955,127.0,Adventure|Drama|Family,Richard Fleischer,Earl Felton| Jules Verne,Kirk Douglas| James Mason| Paul Lukas,English,...,7.2,33152,tt0046672,28200000.0,2.840652e+08,1019.0,"20,000 Leagues Under the Sea (1954)",Adventure|Drama|Sci-Fi,"20,000 Leagues Under the Sea",1954.0


In [11]:
merged_disney_df.isna().sum()

Title                         0
Year                          0
Rated                         0
Released                      0
Runtime                       0
Genre                         0
Director                      1
Writer                        5
Actors                        0
Language                      0
Country                       0
Awards                      101
Metascore                     0
IMDBRating                    0
imdbVotes                     0
imdbID                        0
total_gross                   0
inflation_adjusted_gross      0
movieId                     165
title                       165
genres                      165
movie_title                 165
year                        165
dtype: int64

In [12]:
#genres na?
merged_disney_df.loc[merged_disney_df['genres'].isna()]

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,...,IMDBRating,imdbVotes,imdbID,total_gross,inflation_adjusted_gross,movieId,title,genres,movie_title,year
11,A Bug's Life,1998,G,25 Nov 1998,95.0,Animation|Adventure|Comedy,John Lasseter| Andrew Stanton,John Lasseter| Andrew Stanton| Joe Ranft,Kevin Spacey| Dave Foley| Julia Louis-Dreyfus,English,...,7.2,284819,tt0120623,162798565.0,270635135.0,NaN,NaN,NaN,NaN,NaN
12,A Civil Action,1998,PG-13,08 Jan 1999,115.0,Drama,Steven Zaillian,Jonathan Harr| Steven Zaillian,John Travolta| Robert Duvall| Kathleen Quinlan,English,...,6.6,28711,tt0120633,56709981.0,94274254.0,NaN,NaN,NaN,NaN,NaN
13,A Far Off Place,1993,PG,12 Mar 1993,108.0,Adventure|Drama|Family,Mikael Salomon,Lourens van der Post| Robert Caswell| Jonathan...,Reese Witherspoon| Ethan Embry| Jack Thompson,English,...,6.5,3556,tt0106868,12890752.0,24173059.0,NaN,NaN,NaN,NaN,NaN
14,A Goofy Movie,1995,G,07 Apr 1995,78.0,Animation|Adventure|Comedy,Kevin Lima,Jymn Magon| Chris Matheson| Brian Pimental,Bill Farmer| Jason Marsden| Jim Cummings,English,...,6.9,52129,tt0113198,35336439.0,62828837.0,NaN,NaN,NaN,NaN,NaN
15,The Gun in Betty Lou's Handbag,1992,PG-13,21 Aug 1992,89.0,Comedy|Mystery,Allan Moyle,Grace Cary Bickley,Penelope Ann Miller| Eric Thal| Alfre Woodard,English,...,5.1,1683,tt0104376,3591460.0,6936407.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,Red Rock West,1993,R,14 May 1993,98.0,Crime|Drama|Thriller,John Dahl,John Dahl| Rick Dahl,Nicolas Cage| Dennis Hopper| Lara Flynn Boyle,English,...,7.0,22020,tt0105226,2502551.0,4575683.0,NaN,NaN,NaN,NaN,NaN
612,Endurance,1998,G,14 May 1999,83.0,Biography|Drama|Sport,Leslie Woodhead| Bud Greenspan,NaN,Haile Gebrselassie| Shawananness Gebrselassie|...,English,...,6.3,419,tt0120659,229128.0,372669.0,NaN,NaN,NaN,NaN,NaN
614,Mars Needs Moms,2010,PG,11 Mar 2011,88.0,Animation|Action|Adventure,Simon Wells,Simon Wells| Wendy Wells| Berkeley Breathed,Seth Green| Joan Cusack| Dan Fogler,English,...,5.4,22234,tt1305591,21392758.0,25770522.0,NaN,NaN,NaN,NaN,NaN
615,The Jungle Book 2,2002,G,14 Feb 2003,72.0,Animation|Adventure|Comedy,Steve Trenbirth,Karl Geurs| Carter Crocker| Evan Spiliotopoulos,John Goodman| Haley Joel Osment| Tony Jay,English,...,5.4,16161,tt0283426,47901582.0,70542889.0,NaN,NaN,NaN,NaN,NaN


In [13]:
#genres na?
merged_disney_df.loc[merged_disney_df['genres'] == '(no genres listed)']

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,...,IMDBRating,imdbVotes,imdbID,total_gross,inflation_adjusted_gross,movieId,title,genres,movie_title,year
333,Pirates of the Caribbean: Dead Men Tell No Tales,2017,PG-13,26 May 2017,129.0,Action|Adventure|Fantasy,Joachim Rønning| Espen Sandberg,Jeff Nathanson| Terry Rossio| Ted Elliott,Johnny Depp| Geoffrey Rush| Javier Bardem,English|Spanish,...,6.5,292790,tt1790809,172558876.0,190756685.0,122896.0,Pirates of the Caribbean: Dead Men Tell No Tal...,(no genres listed),Pirates of the Caribbean: Dead Men Tell No Tales,2017.0


In [14]:
#Replace Genre "Not Rated" with "Not-Rated"

merged_disney_df['Rated'] = merged_disney_df['Rated'].str.replace('Not Rated','Not-Rated')

In [15]:
merged_disney_df.isna().sum()

Title                         0
Year                          0
Rated                         0
Released                      0
Runtime                       0
Genre                         0
Director                      1
Writer                        5
Actors                        0
Language                      0
Country                       0
Awards                      101
Metascore                     0
IMDBRating                    0
imdbVotes                     0
imdbID                        0
total_gross                   0
inflation_adjusted_gross      0
movieId                     165
title                       165
genres                      165
movie_title                 165
year                        165
dtype: int64

In [16]:
#movieId na?
merged_disney_df.loc[merged_disney_df['movieId'].isna()]

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,...,IMDBRating,imdbVotes,imdbID,total_gross,inflation_adjusted_gross,movieId,title,genres,movie_title,year
11,A Bug's Life,1998,G,25 Nov 1998,95.0,Animation|Adventure|Comedy,John Lasseter| Andrew Stanton,John Lasseter| Andrew Stanton| Joe Ranft,Kevin Spacey| Dave Foley| Julia Louis-Dreyfus,English,...,7.2,284819,tt0120623,162798565.0,270635135.0,NaN,NaN,NaN,NaN,NaN
12,A Civil Action,1998,PG-13,08 Jan 1999,115.0,Drama,Steven Zaillian,Jonathan Harr| Steven Zaillian,John Travolta| Robert Duvall| Kathleen Quinlan,English,...,6.6,28711,tt0120633,56709981.0,94274254.0,NaN,NaN,NaN,NaN,NaN
13,A Far Off Place,1993,PG,12 Mar 1993,108.0,Adventure|Drama|Family,Mikael Salomon,Lourens van der Post| Robert Caswell| Jonathan...,Reese Witherspoon| Ethan Embry| Jack Thompson,English,...,6.5,3556,tt0106868,12890752.0,24173059.0,NaN,NaN,NaN,NaN,NaN
14,A Goofy Movie,1995,G,07 Apr 1995,78.0,Animation|Adventure|Comedy,Kevin Lima,Jymn Magon| Chris Matheson| Brian Pimental,Bill Farmer| Jason Marsden| Jim Cummings,English,...,6.9,52129,tt0113198,35336439.0,62828837.0,NaN,NaN,NaN,NaN,NaN
15,The Gun in Betty Lou's Handbag,1992,PG-13,21 Aug 1992,89.0,Comedy|Mystery,Allan Moyle,Grace Cary Bickley,Penelope Ann Miller| Eric Thal| Alfre Woodard,English,...,5.1,1683,tt0104376,3591460.0,6936407.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,Red Rock West,1993,R,14 May 1993,98.0,Crime|Drama|Thriller,John Dahl,John Dahl| Rick Dahl,Nicolas Cage| Dennis Hopper| Lara Flynn Boyle,English,...,7.0,22020,tt0105226,2502551.0,4575683.0,NaN,NaN,NaN,NaN,NaN
612,Endurance,1998,G,14 May 1999,83.0,Biography|Drama|Sport,Leslie Woodhead| Bud Greenspan,NaN,Haile Gebrselassie| Shawananness Gebrselassie|...,English,...,6.3,419,tt0120659,229128.0,372669.0,NaN,NaN,NaN,NaN,NaN
614,Mars Needs Moms,2010,PG,11 Mar 2011,88.0,Animation|Action|Adventure,Simon Wells,Simon Wells| Wendy Wells| Berkeley Breathed,Seth Green| Joan Cusack| Dan Fogler,English,...,5.4,22234,tt1305591,21392758.0,25770522.0,NaN,NaN,NaN,NaN,NaN
615,The Jungle Book 2,2002,G,14 Feb 2003,72.0,Animation|Adventure|Comedy,Steve Trenbirth,Karl Geurs| Carter Crocker| Evan Spiliotopoulos,John Goodman| Haley Joel Osment| Tony Jay,English,...,5.4,16161,tt0283426,47901582.0,70542889.0,NaN,NaN,NaN,NaN,NaN


In [17]:
#merged_disney_df['title'] = merged_disney_df['title'].astype(str)

#So after merged, we need to find titles in IMDB lists without it's corresponding MovieLens movieID
#then we take the MovieLens movieId & genres to add into merged_disney_df

for i in merged_disney_df.index:
    if str(merged_disney_df['movieId'][i]) == 'nan':  
        for j in ml_df.index:
            if merged_disney_df['Title'][i] == ml_df['movie_title'][j]:
                print("--------------------------")
                print(merged_disney_df['Title'][i])
                print(ml_df['movie_title'][j])
                merged_disney_df['movieId'][i] = ml_df['movieId'][j]
                merged_disney_df['genres'][i] = ml_df['genres'][j]
                print("--------------------------")

--------------------------
Amy
Amy
--------------------------


/Users/vuvie/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/Users/vuvie/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


--------------------------
Amy
Amy
--------------------------
--------------------------
Black Panther
Black Panther
--------------------------
--------------------------
Black Widow
Black Widow
--------------------------
--------------------------
Black Widow
Black Widow
--------------------------
--------------------------
Captain Marvel
Captain Marvel
--------------------------
--------------------------
A Christmas Carol
A Christmas Carol
--------------------------
--------------------------
A Christmas Carol
A Christmas Carol
--------------------------
--------------------------
Earth
Earth
--------------------------
--------------------------
Earth
Earth
--------------------------
--------------------------
Earth
Earth
--------------------------
--------------------------
Houseguest
Houseguest
--------------------------
--------------------------
I'll Be Home for Christmas
I'll Be Home for Christmas
--------------------------
--------------------------
Night Crossing
Night Crossi

KeyboardInterrupt: 

In [ ]:
merged_disney_df.loc[merged_disney_df['Title'] == 'The Little Mermaid']

In [ ]:
#We are going to add the Genre to genres column if it is nan or (no genres listed) because we will keep genres 
#column (from MovieLens) and drop Genre column from IMDB.
#The reason is MovieLens genres have longer genres list than IMDB

for i in merged_disney_df.index:
    if str(merged_disney_df['title'][i]) == 'nan' or str(merged_disney_df['genres'][i]) =='(no genres listed)':
        print("List of Movies that IMDB Genre nan will be replaced by MovieLens genres")
        print(merged_disney_df['Title'][i])
        print(merged_disney_df['Genre'][i])
        print(merged_disney_df['genres'][i])
        merged_disney_df['genres'][i] = merged_disney_df['Genre'][i]
        print("--------------------------")
        
    elif len(merged_disney_df['Genre'][i]) > len(merged_disney_df['genres'][i]):
        print(merged_disney_df['Genre'][i])
        print(merged_disney_df['genres'][i])
        merged_disney_df['genres'][i] = merged_disney_df['Genre'][i]
        print("--------------------------")

In [ ]:
#This first test list is ready for ML K-means Clustering

merged_disney_df.to_csv (r'output/ML_bb_ready.csv', index = None, header=True)

In [ ]:
movies_df = merged_disney_df['imdbID','Title', 'Year', 'Rated', 'Released', 'Runtime', 'genres']

In [ ]:
revenue_df = ['imdbID', 'total_gross', 'inflation_adjusted_gross']

In [ ]:
#make a running number
revenue_df['rev_id'] = np.arange(len(revenue_df))
#add constant string in front of the running number for Primary Key
revenue_df['rev_id'] = 'r-0' + revenue_df['rev_id'].astype(str)
revenue_df.head()

### Load Completed IMDB TV show dataset

In [ ]:
#Load the complete IMDB dataset
imdb_tv_df = pd.read_csv('input/IMDB_tv.csv')

imdb_tv_df['Rated'] = imdb_tv_df['Rated'].str.replace('Not Rated','Not-Rated')
imdb_tv_df['Title'] = imdb_tv_df['Title'].str.replace('.', '')

imdb_tv_df['Year'] = imdb_tv_df['Year'].str.split('–', 1).str[0]
imdb_tv_df['Year'] = imdb_tv_df['Year'].astype(int)
imdb_tv_df['Genre'] = imdb_tv_df['Genre'].str.replace(',', '|')
imdb_tv_df['Genre'] = imdb_tv_df['Genre'].str.replace(' ', '')

imdb_tv_df['Director'] = imdb_tv_df['Director'].str.replace(',', '|')
#imdb_tv_df['Director'] = imdb_tv_df['Director'].str.replace(' ', '')

imdb_tv_df['Writer'] = imdb_tv_df['Writer'].str.replace(',', '|')
#imdb_tv_df['Writer'] = imdb_tv_df['Writer'].str.replace(' ', '')

imdb_tv_df['Actors'] = imdb_tv_df['Actors'].str.replace(',', '|')
#imdb_tv_df['Actors'] = imdb_tv_df['Actors'].str.replace(' ', '')

imdb_tv_df['Language'] = imdb_tv_df['Language'].str.replace(',', '|')
imdb_tv_df['Language'] = imdb_tv_df['Language'].str.replace(' ', '')

imdb_tv_df['Country'] = imdb_tv_df['Country'].str.replace(',', '|')
#imdb_tv_df['Country'] = imdb_tv_df['Country'].str.replace(' ', '')

imdb_tv_df['imdbVotes'] = imdb_tv_df['imdbVotes'].astype(str)
imdb_tv_df['imdbVotes'] = imdb_tv_df['imdbVotes'].str.replace(',', '')

imdb_tv_df.head()

In [ ]:
#Merge the IMDB tv shows dataset and MovieLens dataset on movie_title & year ONLY
merged_tv_df =pd.merge(imdb_tv_df, ml_df, how='left', left_on=['Title', 'Year'], right_on=['movie_title', 'year'])
merged_tv_df.head()

In [ ]:
#Replace Genre "Not Rated" with "Not-Rated"
merged_tv_df['Rated'] = merged_tv_df['Rated'].str.replace('Not Rated','Not-Rated')

In [ ]:
merged_tv_df.isna().sum()

In [ ]:
#genres na?
merged_tv_df.loc[merged_tv_df['genres'] == '(no genres listed)']

In [ ]:
#genres na?
merged_tv_df.loc[merged_tv_df['Genre'].isna()]

In [ ]:
#So after merged, we need to find titles in IMDB lists without it's corresponding MovieLens movieID
#then we take the MovieLens movieId & genres to add into merged_tv_df

for i in merged_tv_df.index:
    if str(merged_tv_df['movieId'][i]) == 'nan':  
        for j in ml_df.index:
            if merged_tv_df['Title'][i] == ml_df['movie_title'][j]:
                print("--------------------------")
                print(merged_tv_df['Title'][i])
                print(ml_df['movie_title'][j])
                merged_tv_df['movieId'][i] = ml_df['movieId'][j]
                merged_tv_df['genres'][i] = ml_df['genres'][j]
                print("--------------------------")

In [ ]:
merged_tv_df.loc[merged_tv_df['Title'] == 'Lifted']

In [ ]:
merged_tv_df.loc[merged_tv_df['Genre'].isna()]

In [ ]:
#drop the column if Genre is Null

merged_tv_df.dropna(subset = ['Genre'], inplace=True)

In [ ]:
merged_tv_df.loc[merged_tv_df['Genre'].isna()]

In [ ]:
#We are going to add the Genre to genres column if it is nan or (no genres listed) because we will keep genres 
#column (from MovieLens) and drop Genre column from IMDB.
#The reason is MovieLens genres have longer genres list than IMDB

for i in merged_disney_df.index:
    if str(merged_disney_df['title'][i]) == 'nan' or str(merged_disney_df['genres'][i]) =='(no genres listed)':
        print("List of Movies that IMDB Genre nan will be replaced by MovieLens genres")
        print(merged_disney_df['Title'][i])
        print(merged_disney_df['Genre'][i])
        print(merged_disney_df['genres'][i])
        merged_disney_df['genres'][i] = merged_disney_df['Genre'][i]
        print("--------------------------")
        
    elif len(merged_disney_df['Genre'][i]) > len(merged_disney_df['genres'][i]):
        print(merged_disney_df['Genre'][i])
        print(merged_disney_df['genres'][i])
        merged_disney_df['genres'][i] = merged_disney_df['Genre'][i]
        print("--------------------------")  

In [ ]:
merged_tv_df.isna().sum()

In [ ]:
#Make a decision to fill Rated NULL to "G"
merged_tv_df['Rated'].fillna('G', inplace = True)

In [ ]:
merged_tv_df.head()

In [ ]:
#This first test list is ready for ML K-means Clustering

merged_tv_df.to_csv (r'output/ML_tv_ready.csv', index = None, header=True) 

In [ ]:
# #make a running number for column test
# ml_df['test'] = np.arange(len(ml_df))

# #add constant string in front of the running number for Primary Key
# ml_df['test'] = 'wc-00000' + ml_df['test'].astype(str)

# ml_df.head()


In [ ]:
# #check some titles
# merged_disney_df[merged_disney_df.Title.str.startswith('WAL')]

In [ ]:
# #check some titles
# ml_df[ml_df.title.str.startswith('WAL')]

In [ ]:
# #check some titles
# ml_df[ml_df.title.str.startswith('Déjà')]

### Load MovieLens Ratings dataset

In [ ]:
# #load Movie Lens ratings dataset
# ml_ratings_df = pd.read_csv('input/ratings.csv')
# ml_ratings_df.head()

In [ ]:
# ml_ratings_df.loc[ml_ratings_df['movieId'] == 49278]

In [ ]:
# ml_ratings_df[ml_ratings_df.movieId.str.startswith('49278')]